In [1]:
import os
from tqdm.auto import tqdm

import numpy as np
import pandas as pd
import pyvista as pv
import potpourri3d as pp3d

import torch
from torch_geometric.data import Data

In [ ]:
class DECModule:
    """Differential Exterior Calculus (DEC) module utilizing precomputed DEC operators."""
    @staticmethod
    def oneform(field: torch.tensor, data: Data):
        return torch.einsum('ij,ij->i', (0.5 * field[data.edge_faces_index].sum(axis=1)), data.edge_vectors, optimize=True)
    
    @staticmethod
    def curl(oneform: torch.tensor, data: Data):
        return data.dec_hodge2 * data.dec_d1 * oneform

    @staticmethod
    def divergence(oneform: torch.tensor, data: Data):
        return data.dec_hodge0Inverse * data.dec_d0.T * data.dec_hodge1 * oneform
    

class ComputeHHD:
    def __init__(self, y: str):
        self.y = y
    
    def _decompose(self, points: torch.tensor, faces: torch.tensor, field: torch.tensor):
        dec = pp3d.DifferentialExteriorCalculus(points.numpy(), faces.numpy())
        decomposition = {"exact": [], "coexact": [], "harmonic": []}

        for bc in field.size(1):
            bc_decomposition = {"exact": [], "coexact": [], "harmonic": []}

            for t in field.size(2):
                one_form = dec.to_1Form(field[:, bc, t])
                exact, coexact, harmonic = dec.hodge_decomposition(one_form)

                exact = dec.interpolate_1Form(exact)
                coexact = dec.interpolate_1Form(coexact)
                harmonic = dec.interpolate_1Form(harmonic)

                bc_decomposition["exact"].append(exact)
                bc_decomposition["coexact"].append(coexact)
                bc_decomposition["harmonic"].append(harmonic)

            decomposition["exact"].append(bc_decomposition["exact"])
            decomposition["coexact"].append(bc_decomposition["coexact"])
            decomposition["harmonic"].append(bc_decomposition["harmonic"])

        decomposition_torch = {f"{self.y}_{k}": torch.tensor(v) for k, v in decomposition.items()}
        operators = {
            "edge_faces_index": torch.from_numpy(dec.edge_faces),
            "edge_vectors": torch.from_numpy(dec.edge_vectors),
            "dec_hodge0Inverse": torch.from_numpy(dec.hodge0Inverse),
            "dec_hodge1": torch.from_numpy(dec.hodge1),
            "dec_hodge2": torch.from_numpy(dec.hodge2),
            "dec_d0": torch.from_numpy(dec.d0),
            "dec_d1": torch.from_numpy(dec.d1)
        }

        return decomposition_torch | operators
        
    def __call__(self, data: Data):
        points, face = data.pos, data.face.T
        field = data[self.y]

        hhd_fields = self._decompose(points, face, field)
        
        for name, field in hhd_fields.items():
            data[name] = field

        return data

In [2]:
# Load mesh & field
mesh = pv.read("/Users/patrykrygiel/Downloads/hdd-cell-t008.vtp")
field = mesh["wss"]

# Perform HHD
dec = pp3d.DifferentialExteriorCalculus(mesh.points, mesh.regular_faces)
one_form = dec.to_1Form(field)
exact, coexact, harmonic = dec.hodge_decomposition(one_form)
hhd_dict = {"exact": exact, "coexact": coexact, "harmonic": harmonic}

# Assemble measure scores
scores = []

for name, one_form in hhd_dict.items():
    one_form_interp = dec.to_1Form(dec.interpolate_1Form(one_form))

    # Exact HHD
    scores.append([
        name, 
        False,
        np.mean(dec.divergence(one_form)),
        np.mean(dec.curl(one_form)),
    ])

    # Interpolated & back
    scores.append([
        name,
        True,
        np.mean(dec.divergence(one_form_interp)),
        np.mean(dec.curl(one_form_interp)),
    ])

scores_df = pd.DataFrame(scores, columns=["component", "interpolated", "divergence", "curl"]).set_index(["interpolated", "component"]).sort_index()
scores_df

divergence          curl
interpolated component                            
False        coexact   -9.616415e-15 -1.444531e+01
             exact      1.282386e+01 -5.432033e-16
             harmonic   1.677010e-04 -1.547304e-12
True         coexact    4.800672e-01 -1.396759e+01
             exact      1.282386e+01 -3.542581e-17
             harmonic   1.677010e-04 -1.481661e-12

In [4]:
# Save onto mesh
mesh["exact"] = dec.interpolate_1Form(exact)
mesh["coexact"] = dec.interpolate_1Form(coexact)
mesh["harmonic"] = dec.interpolate_1Form(harmonic)
mesh["wss-rec"] = mesh["exact"] + mesh["coexact"] + mesh["harmonic"]
mesh["wss-mae"] = np.abs(mesh["wss"] - mesh["wss-rec"])
mesh.save("hhd-py.vtp")

print(f"\nDecomposition MAE: {mesh['wss-mae'].mean()}")


Decomposition MAE: 0.24445975610710413


In [7]:
def hdd_dir(dirpath: str):
    os.makedirs(out_dir := (dirpath + "-hdd"), exist_ok=True)
    timesteps = sorted(os.listdir(dirpath))
    dec = None

    for timestep in tqdm(timesteps):
        mesh = pv.read(os.path.join(dirpath, timestep))
        points, faces = mesh.points, mesh.regular_faces

        # Interpolate field onto faces
        point_field = mesh["y_true"]
        face_field = point_field[faces].mean(axis=1)

        if dec is None:
            dec = pp3d.DifferentialExteriorCalculus(points, faces)

        one_form = dec.to_1Form(face_field)
        exact, coexact, harmonic = dec.hodge_decomposition(one_form)

        mesh["coexact-curl"] = dec.interpolate_1Form(dec.curl(coexact))
        mesh["coexact-div"] = dec.interpolate_1Form(dec.divergence(coexact))

        mesh["wss"] = face_field
        mesh["exact"] = dec.interpolate_1Form(exact)
        mesh["coexact"] = dec.interpolate_1Form(coexact)
        mesh["harmonic"] = dec.interpolate_1Form(harmonic)
        mesh["wss-rec"] = mesh["exact"] + mesh["coexact"] + mesh["harmonic"]
        mesh["wss-ae"] = np.abs(mesh["wss"] - mesh["wss-rec"])
        mesh.save(os.path.join(out_dir, timestep))

        print(f"Decomposition MAE: {mesh['wss-ae'].mean()}")

hdd_dir("/Users/patrykrygiel/Documents/UTWENTE/Projects/VASCUL-AID/vortex")

  0%|          | 0/21 [00:00<?, ?it/s]

Decomposition MAE: 0.03879191513555843
Decomposition MAE: 0.09932909877402935
Decomposition MAE: 0.18614234333505256
Decomposition MAE: 0.29169141513743435
Decomposition MAE: 0.37417381293155033
Decomposition MAE: 0.38752147208746385
Decomposition MAE: 0.3263218675083875
Decomposition MAE: 0.22711804719252732
Decomposition MAE: 0.13871833314752707
Decomposition MAE: 0.08697999488143413
Decomposition MAE: 0.06167604933765465
Decomposition MAE: 0.0537570549147973
Decomposition MAE: 0.059802733110568086
Decomposition MAE: 0.07005674095980456
Decomposition MAE: 0.07115205712046112
Decomposition MAE: 0.062463382887151725
Decomposition MAE: 0.05191172281820278
Decomposition MAE: 0.04357220796128482
Decomposition MAE: 0.043391286168440506
Decomposition MAE: 0.04211536570513164
Decomposition MAE: 0.03957312274019295
